# TP 12 Simulación - Call Center Metrogas

## Contexto
Se dispone de 3 tipos de puestos, y diferentes cantidades de cada uno de ellos:

- Emergencia (E)
- Comercial (C)
- Polifuncional (P)

Y dos colas:

- Cola para emergencia (NSE)
- Cola para comercial (NSC)

Los puestos comerciales y los puestos de emergencia solo atienden su propia cola. Los puestos polifuncionales atienden solo cuando todos los puestos comerciales y/o todos los puestos de emergencia estan ocupados y hay gente esperando en la cola para ser atendido,sera atendido por un polifuncional, dandole mas prioridad a los que estan en la cola de emergencia 

## Importando librerias de simpy

In [11]:
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections
from itertools import combinations_with_replacement

_____

## Simulacion de una llegada

In [2]:
def distribucion_llegada(env,puestos,cont,n):
    while True:
    #for i in range(n):
    
        cont["NT"] += 1
        
        # Genero un cliente random 
        r = random.random()
        #print("Distribucion de generacion de tipo de cliente: %5.2f"%r)
        
        # Si genere un cliente para emergencia
        if r <= cont["PE"]:
            
            ce = Customer(name = "clienteE%02d"%cont["NT"])
            
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = generar_IA(cont)
            yield env.timeout(ia)
            #print("T=%5.2f %s: COLA EMERGENCIA - llegue con un IA: %5.2f"%(env.now,ce.name,ia))
            colas(puestos) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(ce.llamada_emergencia(env,puestos,cont))
            
        # Si genere un cliente para comercial    
        else: 
            
            cc = Customer(name = "clienteC%02d"%cont["NT"])
            # Genero IA para el siguiente cliente y genero el timeout para su llegada 
            ia = generar_IA(cont)
            yield env.timeout(ia)
            #print("T=%5.2f %s: COLA COMERCIAL - llegue con un IA: %5.2f"%(env.now,cc.name,ia))
            colas(puestos) # Imprimo el estao de las colas
            
            # Genero la llamada
            env.process(cc.llamada_comercial(env,puestos,cont))

In [3]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        emergencia, polifuncional = puestos[1].request(priority=0), puestos[2].request(priority=0)
        atendido = yield emergencia | polifuncional 
        #assert atendido == {emergencia}
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            #print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto polifuncional despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            #print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            #print("T=%5.2f, %s: Sali de un puesto de polifuncional"%(env.now,self.name))
        
        # Me atendieron por un puesto de emergencia
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            #print("T=%5.2f, %s: COLA EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            #print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            #print("T=%5.2f, %s: Sali de un puesto de emergencia"%(env.now,self.name))
            
            
    def llamada_comercial(self,env,puestos,cont):
        
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        # puestos = [comercial, emergencia, polifuncional]
        comercial, polifuncional = puestos[0].request(priority=0), puestos[2].request(priority=1) # comercial tn menos prioridad q emergencia
        atendido = yield comercial | polifuncional 
        
        # Me atendieron por puestos polifuncionales?
        if polifuncional in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
                
            ta = generar_TA(cont)
            #print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto polifuncional despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            #print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            #print("T=%5.2f, %s: Sali de un puesto de polifuncional"%(env.now,self.name))
        
        # Me atendieron por un puesto comercial
        else:
            wait = env.now-arrive #Tiempo de espera
            
            if wait <= 40:
                cont["NEC"] += 1
            
            ta = generar_TA(cont)
            #print("T=%5.2f, %s: COLA COMERCIAL - Me atendieron en un puesto comercial despues de esperar %5.2f"%(env.now,self.name,wait))
            colas(puestos)
            #print("Tiempo de Atencion generado para %s: %5.2f"%(self.name,ta))
            yield env.timeout(ta)
            #print("T=%5.2f, %s: Sali de un puesto de comercial"%(env.now,self.name))

In [4]:
def colas(p):
    #print ("Atendiendo:[c:%1d, e:%1d, p:%1d]"%(p[0].count,p[1].count,p[2].count))
    #print ("En cola:[c:%1d, e:%1d]"%(len(p[0].queue),len(p[1].queue)))


In [5]:
def generar_TA(cont):
    r = 0

    # Mientras caiga fuera de esta zona, generame otro
    while r < 52.2 or r > 439.2:

        try:
            r = int(random.lognormvariate(cont["muTA"],cont["sigmaTA"]))
        except OverflowError:
            r = float('inf')
    
    return r

In [6]:
def generar_IA(cont):
    r = 0

    while r <= 0 or r > 52.36:

        try:
            r = int(random.lognormvariate(cont["muIA"],cont["sigmaIA"]))
        except OverflowError:
            r = float('inf')
    
    return r

## Inicializacion de la simulacion

In [10]:
def simular(cantCom,cantEmer,cantPolif,ci,maxClient,maxTime):
    env = simpy.Environment()
    puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer),
           simpy.PriorityResource(env,capacity=cantPolif)] # menor el numero priority, mayor es la prioridad
           
    env.process(distribucion_llegada(env,puestos,ci,n=maxClient))
    env.run(until=maxTime)
    
    #print("===========================================================================")
    #print("RESULTADOS:")
    #print("Clientes totales: %1d"%ci["NT"])
    #print("Tiempo de espera en cola total: %1d"%ci["NEC"]) # Sacar esto despues
    PEC = (ci["NEC"] / ci["NT"]) * 100
    # Porcentaje de espera menor a 40 segundos
    #print("Porcentaje de espera en cola menor a 40 segundos: %5.2f"%PEC)
    # Objetio: Mayor o igual al 90%

    #print('Cantidad de puestos comerciales',cantCom)
    #print('Cantidad de puestos de emergencia',cantEmer)
    #print('Cantidad de puestos polifuncionales',cantPolif)
    #print("Cantidad de gente atendido por polifuncional",ci["multitasking"])

## Condiciones Iniciales

In [8]:
# Experimental data
NT = 0
maxTime = 14400 #segundos
maxClient = 10000
#random.seed(RANDOM_SEED)
PE = 0.0619 #Porcentaje Emergencia
PC = 0.9381 #Porcentaje Comercial
NEC = 0 #Cantidad de gente que espero en cola menor a 40 segundos
PEC = 0.0 #Porcentaje de espera en cola menor a 40 segundos

# TA - distribucion normal
muTA = 245.712
sigmaTA = 193.499

# IA - distribucion normal
muIA = 9.585 
sigmaIA = 42.785

# Contadores
ci = {"multitasking":0,
     "muTA":muTA,
     "sigmaTA":sigmaTA,
     "muIA":muIA,
     "sigmaIA":sigmaIA,
     "PE":PE,
     "PC":PC,
     "emer":0,
     "com":0,
     "pol":0,
     "NEC":NEC,
     "NT":NT}

## Ingreso manual de Puestos de atencion - variables de control

!! Generar una version para automatizar esto, que genere diferentes combinaciones dado un rango

Y tmbn dejar la opcion de generarlo manualmente

In [19]:

print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
ci["com"] = cantCom
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())
ci["emer"] = cantEmer
print('Ingrese cantidad de puestos polifuncionales')
cantPolif = int(input())
ci["pol"] = cantPolif


Ingrese cantidad de puestos comerciales
60
Ingrese cantidad de puestos de emergencia
50
Ingrese cantidad de puestos polifuncionales
80


## Ejecucion automatica con combinaciones de variables de control

In [23]:

for i in range(20,110,10):
    
    n, k = [i-10,i], 3
    cant_puestos = list(combinations_with_replacement(n,k))
    
    for ii in range(4):
        cantCom = cant_puestos[ii][0]
        cantEmer = cant_puestos[ii][1]
        cantPolif = cant_puestos[ii][2]
        iniciar_sim(cantCom,cantEmer,cantPolif,ci,maxClient,maxTime)
        

IndexError: tuple index out of range